In [57]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
import pandas as pd
import numpy as np

from src.features import build_features

import glob

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]
COLUMNAS_POBLACION_FONASA = ["2018", "2019", "2020", "2021", "2022", "2023"]

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla De Pascua",
]

COMUNAS_SSMC = ["Santiago", "Maipú", "Estación Central", "Cerrillos"]
COMUNAS_SSMS = [
    "San Bernardo",
    "Pedro Aguirre Cerda",
    "La Cisterna",
    "San Joaquín",
    "Lo Espejo",
    "San Miguel",
    "Paine",
    "Calera De Tango",
    "Buin",
    "El Bosque",
]

## Obtener poblacion INE y FONASA


In [75]:
df_ine = pd.read_csv("../data/processed/df_ine.csv")
df_fonasa = pd.read_csv("../data/processed/df_fonasa.csv", dtype={"ANO_INFORMACION": str})

## Definición de Estratos a Calcular

En este cuadernillo se quiere calcular la población para distintos Estratos y Grupos Etarios del País. Para esto, se utilizará la base de datos de la estimación de la población del Instituto Nacional de Estadísticas (INE) entre 2017 y 2035.

En primer lugar, se definirán los siguientes estratos a calcular:

1. País
2. Regiones del País
3. SSMO
4. SSMC
5. SSMS


### INE


In [76]:
ESTRATOS_A_CALCULAR_INE = {}

# Obtiene estratos a calcular
df_pais = df_ine.copy()
ESTRATOS_A_CALCULAR_INE["Pais"] = df_pais

for region in sorted(df_pais["M"].unique()):
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR_INE[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR_INE["SSMO"] = df_ssmo

df_ssmc = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMC)]
ESTRATOS_A_CALCULAR_INE["SSMC"] = df_ssmc

df_ssms = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMS)]
ESTRATOS_A_CALCULAR_INE["SSMS"] = df_ssms

### FONASA


In [77]:
# Obtiene estratos a calcular FONASA
ESTRATOS_A_CALCULAR_FONASA = {}

# Estrato pais
df_pais_fonasa = df_fonasa.copy()
ESTRATOS_A_CALCULAR_FONASA["Pais"] = df_pais_fonasa

# Por region
for region in sorted(df_pais_fonasa["REGION"].fillna("").unique()):
    if region:
        df_region_fonasa = df_pais_fonasa.query("REGION == @region").copy()
        ESTRATOS_A_CALCULAR_FONASA[region] = df_region_fonasa

# Obtiene SSMO y SSMC
df_ssmo_fonasa = df_pais_fonasa[df_pais_fonasa["COMUNA"].isin(COMUNAS_SSMO)]
df_ssmc_fonasa = df_pais_fonasa[df_pais_fonasa["COMUNA"].isin(COMUNAS_SSMC)]
df_ssms_fonasa = df_pais_fonasa[df_pais_fonasa["COMUNA"].isin(COMUNAS_SSMS)]

ESTRATOS_A_CALCULAR_FONASA["SSMO"] = df_ssmo_fonasa
ESTRATOS_A_CALCULAR_FONASA["SSMC"] = df_ssmc_fonasa
ESTRATOS_A_CALCULAR_FONASA["SSMS"] = df_ssms_fonasa

## Definición de grupos etarios a calcular

En el próximo diccionario se definirán todos los grupos etarios que se quieren calcular para cada estrato previamente definidos.


In [78]:
# Define todos los grupos etarios a calcular para INE.
query_strings_ine = {
    "todos": "",  # Todo el pais
    "hombres": "hombre_mujer == 1",  # Hombres
    "mujeres": "hombre_mujer == 2",  # Mujeres
    "recien_nacidos_vivos": "Edad == 0",  # Recien Nacidos Vivos
    "sobre_15": "Edad >= 15",  # Sobre 15 anios
    "sobre_20": "Edad >= 20",  # Sobre 20 anios
    "sobre_45": "Edad >= 45",  # Sobre 45 anios
    "sobre_60": "Edad >= 60",  # Sobre 60 anios
    "sobre_65": "Edad >= 65",  # Sobre 65 anios
    "sobre_75": "Edad >= 75",  # Sobre 75 anios
    "entre_15_a_24_anios": "Edad >= 15 and Edad <= 24",  # Entre 15 y 24 anios
    "entre_25_a_44_anios": "Edad >= 25 and Edad <= 44",  # Entre 25 y 44 anios
    "entre_45_a_59_anios": "Edad >= 45 and Edad <= 59",  # Entre 45 y 59 anios
    "entre_60_y_70": "Edad >= 60 and Edad <= 70",  # Entre 60 y 70 anios
    "entre_60_a_74_anios": "Edad >= 60 and Edad <= 74",  # Entre 60 a 74 anios
}

# Define todos los grupos etarios a calcular para FONASA
query_strings_fonasa = {
    "todos": "",  # Todo el pais
    "hombres": "SEXO == 'HOMBRE'",  # Hombres
    "mujeres": "SEXO == 'MUJER'",  # Mujeres
    "recien_nacidos_vivos": "",  # Recien Nacidos Vivos
    "sobre_15": "EDAD_TRAMO >= 15",  # Sobre 15 anios
    "sobre_20": "EDAD_TRAMO >= 20",  # Sobre 20 anios
    "sobre_45": "EDAD_TRAMO >= 45",  # Sobre 45 anios
    "sobre_60": "EDAD_TRAMO >= 60",  # Sobre 60 anios
    "sobre_65": "EDAD_TRAMO >= 65",  # Sobre 65 anios
    "sobre_75": "EDAD_TRAMO >= 75",  # Sobre 75 anios
    "entre_15_a_24_anios": "EDAD_TRAMO >= 15 and EDAD_TRAMO < 25",  # Entre 15 y 24 anios
    "entre_25_a_44_anios": "EDAD_TRAMO >= 25 and EDAD_TRAMO < 45",  # Entre 25 y 44 anios
    "entre_45_a_59_anios": "EDAD_TRAMO >= 45 and EDAD_TRAMO < 60",  # Entre 45 y 59 anios
    "entre_60_y_70_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO <= 70",  # Entre 60 y 70 anios
    "entre_60_a_74_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO < 75",  # Entre 60 a 74 anios
    "mayores_a_75_anios": "EDAD_TRAMO >= 75",  # Mayores a 75 anios
}

Finalmente, se calculará la cantidad de población para distintos estratos y grupos etarios.


In [79]:
# Obtiene poblaciones INE por estratos y grupos etarios
poblaciones_ine = build_features.procesar_resultados_por_estrato_y_grupos_etarios(
    ESTRATOS_A_CALCULAR_INE, query_strings_ine, COLUMNAS_POBLACION_INE, "INE"
)

# Obtiene poblaciones FONASA por estratos y grupos etarios
poblaciones_fonasa = build_features.procesar_resultados_por_estrato_y_grupos_etarios(
    ESTRATOS_A_CALCULAR_FONASA, query_strings_fonasa, "CUENTA_BENEFICIARIOS", "FONASA"
)

## Obtener porcentaje de población FONASA total

En este caso, se quiere saber el % de población FONASA a nivel Nacional. Esto se utilizará para obtener los casos FONASA al 2035.


In [80]:
porcentaje_fonasa = build_features.obtener_porcentaje_de_fonasa_pais(
    poblaciones_ine,
    poblaciones_fonasa,
    COLUMNAS_POBLACION_FONASA,
)

In [81]:
print("Los porcentajes de FONASA son:")
display(round(porcentaje_fonasa * 100, 1))

Los porcentajes de FONASA son:


,2018,2019,2020,2021,2022,2023,suma_poblacion,2018,2019,2020,...,2022,2023,suma_poblacion,2018,2019,2020,2021,2022,2023,porcentaje_fonasa_acumulado
Estrato,,,,,,,,,,,,,,,,,,,,,
Pais,1875140500,1910721600,1945831000,1967836300,1982856300,1996088900,11678474600,1389961300,1462876000,1491435900,...,1544786400,1605825400,8998747600,74.1,76.6,76.6,76.4,77.9,80.4,77.1
Antofagasta,64502200,66856300,69185400,70374600,70963700,71414200,413296400,41373100,44261300,45794400,...,49532900,52766400,280686500,64.1,66.2,66.2,66.7,69.8,73.9,67.9
Arica y Parinacota,24190100,24703600,25211000,25538000,25772200,25980200,151395100,18927000,20089000,20578700,...,22372000,23206000,126450100,78.2,81.3,81.6,83.3,86.8,89.3,83.5
Atacama,30783500,31130700,31470900,31673700,31800400,31904800,188764000,24093500,25207100,25737700,...,26833500,28027000,155891600,78.3,81.0,81.8,82.1,84.4,87.8,82.6
Aysén del General Carlos Ibáñez del Campo,10602300,10668000,10729700,10773700,10804700,10830600,64409000,7561700,7950100,8112500,...,8241500,8408300,48408600,71.3,74.5,75.6,75.5,76.3,77.6,75.2
Biobío,164546000,165474400,166369600,167059000,167626900,168143000,999218900,133156100,137390000,139116600,...,141999600,146118400,837329100,80.9,83.0,83.6,83.5,84.7,86.9,83.8
Coquimbo,80721300,82172600,83609600,84807900,85876900,86910300,504098600,64813700,68324800,69993500,...,72596000,75048500,421554600,80.3,83.1,83.7,83.5,84.5,86.4,83.6
La Araucanía,100142000,100796500,101434300,101954800,102402900,102820100,609550600,84477100,87938900,89945300,...,93065100,95786000,542086100,84.4,87.2,88.7,89.1,90.9,93.2,88.9
Libertador General Bernardo O'Higgins,96648600,97886800,99106300,100095900,100955200,101770100,596462900,77352400,80665000,82619600,...,85762000,88799900,498224900,80.0,82.4,83.4,82.9,85.0,87.3,83.5


En este apartado se leerán las incidencias, junto a las poblaciones respectivas donde se tienen que aplicar tales incidencias. Posteriormente, se calcularán los casos teóricos para cada problema de salud, y para cada estrato definido.


## Lectura de Incidencias por problema de salud


In [82]:
RUTA_PLANILLA_INCIDENCIAS = "../data/external/incidencias_y_prevalencias_INT.xlsx"
COLUMNAS_A_UTILIZAR = [
    "Diagnostico",
    "Diagnosticos Contenidos",
    "Estadística",
    "Casos (Cada 100.000)",
    "Edad Incidencia",
    "Área de Influencia",
    "Porcentaje Hospitalizados de Ambulatorios",
]


# Lee planilla de incidencias
incidencias = pd.read_excel(RUTA_PLANILLA_INCIDENCIAS, usecols=COLUMNAS_A_UTILIZAR)

# Filtra los diagnosticos limitados por oferta
limitados_por_oferta = incidencias.query("`Estadística` == 'Acotado por oferta'")

# Convierte las incidencia a numeros e imputa las faltantes como NaN
incidencias["Casos (Cada 100.000)"] = pd.to_numeric(
    incidencias["Casos (Cada 100.000)"], errors="coerce"
)

# Obtiene la fraccion de la incidencia, para multiplicar con la poblacion directamente
incidencias["rate_incidencia"] = incidencias["Casos (Cada 100.000)"] / 100000

# Corrige prevalencias en 5 anios
idx_prevalencias = incidencias["Estadística"] == "Prevalencia"
incidencias.loc[idx_prevalencias, "rate_incidencia"] = (
    incidencias.loc[idx_prevalencias, "rate_incidencia"] / 5
)

## Estimacion de casos con INE


In [83]:
# Une las incidencias con las poblacion calculadas segun los estratos respectivos
casos_INE = incidencias.merge(poblaciones_ine, how="left", on="Edad Incidencia")

# Multiplica la incidencia por las poblaciones
casos_INE[COLUMNAS_POBLACION_INE] = casos_INE[COLUMNAS_POBLACION_INE].multiply(
    casos_INE["rate_incidencia"], axis="index"
)

## Estimacion de casos con FONASA


In [84]:
# Une las incidencias con las poblacion calculadas segun los estratos respectivos
casos_FONASA = incidencias.merge(poblaciones_fonasa, how="left", on="Edad Incidencia")

# Multiplica la incidencia por las poblaciones
casos_FONASA[COLUMNAS_POBLACION_FONASA] = casos_FONASA[COLUMNAS_POBLACION_FONASA].multiply(
    casos_FONASA["rate_incidencia"], axis="index"
)

## Estimacion de casos FONASA, utilizando la poblacion INE y el % historico de la poblacion FONASA


In [85]:
# Copia los casos INE ya calculados
casos_FONASA_desde_INE = casos_INE.copy()

# Agrega el porcentaje FONASA de cada uno de los estratos calculados
casos_FONASA_desde_INE = casos_FONASA_desde_INE.merge(
    porcentaje_fonasa["porcentaje_fonasa_acumulado"],
    how="left",
    left_on="Estrato",
    right_index=True,
)

# # Multiplica el % de FONASA con la poblacion INE
casos_FONASA_desde_INE[COLUMNAS_POBLACION_INE] = casos_FONASA_desde_INE[COLUMNAS_POBLACION_INE].mul(
    casos_FONASA_desde_INE["porcentaje_fonasa_acumulado"], axis=0
)

## Acotación de casos según área de influencia para el INT

Una vez obtenido los casos proyectados hasta el 2035, se procederá a acotar cada uno de los diagnósticos y sus casos según su área de influencia determinada.


In [86]:
area_de_infl_INT = casos_FONASA_desde_INE.copy()

# Separa las areas de influencia por comas, para dejarlas en una lista
area_de_infl_INT["Área de Influencia"] = area_de_infl_INT["Área de Influencia"].str.split(",")
area_de_infl_INT = area_de_infl_INT.explode("Área de Influencia")
area_de_infl_INT["Área de Influencia"] = area_de_infl_INT["Área de Influencia"].str.strip()

# Solamente deja las filas donde el Area de Influencia coincida con el Estrato
area_de_infl_INT = (
    area_de_infl_INT.query("`Área de Influencia` == `Estrato`")
    .groupby("Diagnostico")[COLUMNAS_POBLACION_INE]
    .sum()
)
# Une el resultado de casos por area de influencia a las incidencias iniciales
area_de_infl_INT = (
    incidencias.set_index("Diagnostico")
    .merge(area_de_infl_INT, how="left", left_index=True, right_index=True)
    .reset_index()
)

# Elimina los diagnosticos acotados por oferta calculados incorrectamente
area_de_infl_INT = area_de_infl_INT.query("`Estadística` != 'Acotado por oferta'")

In [87]:
# Obtiene los casos de los diags acotado por oferta
area_de_infl_INT_acotados_por_oferta = limitados_por_oferta.copy()
for anio_ine in COLUMNAS_POBLACION_INE:
    area_de_infl_INT_acotados_por_oferta[anio_ine] = area_de_infl_INT_acotados_por_oferta[
        "Casos (Cada 100.000)"
    ]

In [88]:
# Une los casos acotados por oferta a los casos por incidencia y prevalencia
area_de_infl_INT = pd.concat([area_de_infl_INT, area_de_infl_INT_acotados_por_oferta])

In [89]:
dfs_a_guardar = {
    "poblacion_INE": poblaciones_ine,
    "poblacion_FONASA": poblaciones_fonasa,
    "porcentaje_FONASA_por_estrato": porcentaje_fonasa.reset_index(),
    "casos_teoricos_INE": casos_INE,
    "casos_teoricos_FONASA_desde_INE": casos_FONASA_desde_INE,
    "casos_area_de_influencia_INT": area_de_infl_INT,
}

with pd.ExcelWriter("../data/interim/casos_teoricos_diagnosticos.xlsx") as writer:
    for sheet_name, df_a_guardar in dfs_a_guardar.items():
        try:
            df_a_guardar = df_a_guardar.drop(columns="rate_incidencia")
        except KeyError:
            pass

        df_a_guardar.to_excel(writer, sheet_name=sheet_name, index=False)